In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from h3 import h3
import json
from urllib.request import URLError, Request, urlopen
from itertools import combinations
from itertools import permutations
from dateutil import parser
from datetime import datetime, timedelta
import math
import networkx as nx

import csv
file = open('log.csv','a')

def get(dataframe,trip_type):
    a,b,c,d=[],[],[],[]
    df_distance =  pd.DataFrame(columns = ['pickup_h3','dropoff_h3','distance','duration'])
    for node_a, node_b in list(permutations(dataframe.index, 2)):
        temp_curr, temp_next = [], []
        
        if trip_type==2:
            h3 = 'pickup_h3'
            lat_long = 'pickup'
        else:
            h3 = 'dropoff_h3'
            lat_long = 'dropoff'
        from_location = dataframe.iloc[node_a][h3]
        to_location = dataframe.iloc[node_b][h3]
        u1,u2,v1,v2= dataframe.iloc[node_a][lat_long+'_latitude'], dataframe.iloc[node_a][lat_long+'_longitude'],\
        dataframe.iloc[node_b][lat_long+'_latitude'],dataframe.iloc[node_b][lat_long+'_longitude']
        
        request_str = 'http://localhost:8989/route?point=' + str(u1) + '%2C' + str(u2) + '&point=' + str(
            v1) + '%2C' + str(v2) + '&vehicle=car'
        request = Request(request_str)
        res = requests.get(request_str)
        if 'paths' in json.loads(res.text):
            distance = json.loads(res.text)['paths'][0]['distance']

            time = json.loads(res.text)['paths'][0]['time']
            minute, msec = divmod(time, 60000)
            if (distance / 1609.344) <=2:
                a.append(from_location)
                b.append(to_location)
                c.append(distance / 1609.344)  # convert meters to miles
                d.append(minute + (msec / 100000))  # convert ms to s and add to min
        else:
            continue

    df_distance['pickup_h3'] = a
    df_distance['dropoff_h3'] = b
    df_distance['distance'] = c
    df_distance['duration'] = d
    return df_distance

In [22]:
from tqdm import tqdm
def get_distance(df_sub,trip_type):
    final_distance=[]
    for _,trips in tqdm(df_sub.groupby(['pool_window']),total=df_sub.groupby(['pool_window']).ngroups):
        trips = trips.reset_index()
        df_distance=  get(trips,trip_type)
        final_distance.append(df_distance)
    
    df_distance = pd.concat(final_distance)
    df_distance.drop_duplicates(subset=['pickup_h3','dropoff_h3'],keep=False,inplace=True)
    return df_distance

# Reload the respective month data

In [28]:
def ceil_dt(dt, delta):
        return datetime.min + math.ceil((dt - datetime.min) / delta) * delta
trip_file = input("Enter the full path to the month's trip data:")
trip_type = input("Enter the trip_type\n 1.From LGA\n 2.To LGA\n")
if trip_type =='1':
    resolution = (15,15)
    distance_file = 'LGA_as_Pickup_months'
else:
    resolution = (10,8)
    distance_file = 'LGA_drop_off_months'
df = pd.read_csv(trip_file)
columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime','passenger_count',\
       'trip_distance', 'pickup_longitude','pickup_latitude','dropoff_longitude', 'dropoff_latitude']
df = df[columns]
df.rename(columns={'tpep_pickup_datetime':'pickup_time',
       'tpep_dropoff_datetime':'dropoff_time'},inplace=True)
drop_index=df[(df.pickup_latitude==0)|(df.pickup_longitude==0)].index
df.drop(drop_index,inplace=True)
df['pickup_time'] = pd.to_datetime(df['pickup_time'])
df['dropoff_time'] = pd.to_datetime(df['dropoff_time'])
df['pickup_h3'] = df.apply(lambda x: h3.geo_to_h3(x['pickup_latitude'], x['pickup_longitude'], resolution[0]), axis=1)
df['dropoff_h3'] = df.apply(lambda x: h3.geo_to_h3(x['dropoff_latitude'], x['dropoff_longitude'], resolution[1]), axis=1)
df['duration'] = (df['pickup_time']-df['dropoff_time']).dt.seconds
df['delay'] = df['duration'].apply(lambda x: x*0.20)
df.shape

Enter the full path to the month's trip data:/home/shet/Documents/CS_RideSharing/Data/LGA as dropoff/LaGuardia_as_dropoff_2015-Jan.csv
Enter the trip_type
 1.From LGA
 2.To LGA
2


(53625, 12)

In [31]:
from time import time
month = 1 # the month you want to download
for day in range(1,32):
    start = time()
    if day <= 9:
        start_date ='2015-0'+str(month)+'-0'+str(day)+' 00:00:00'
        end_date = '2015-0'+str(month)+'-0'+str(day)+' 23:59:59'
        
    else:
        start_date = '2015-0'+str(month)+'-'+str(day)+' 00:00:00'
        end_date = '2015-0'+str(month)+'-'+str(day)+' 23:59:59'
    
    df_sub = df[(df['pickup_time'] >= start_date) & (df['dropoff_time'] <= end_date)]
    df_sub.reset_index(drop=True,inplace=True)
    
    df_sub['pool_window'] = df_sub['pickup_time'].apply(lambda x: ceil_dt(x.to_pydatetime(), timedelta(minutes=10)))
    print("Starting distance calculation...")
    df_distance = get_distance(df_sub,int(trip_type))
    df_distance.reset_index(drop=True,inplace=True) 
    print('Time take for 1 day = {} min'.format((time()-start)/60))
    file_name = 'Distance/'+distance_file+'/'+start_date.split()[0]+'.csv'
    df_distance.to_csv(file_name)
    print('Done with {} data\n'.format(file_name))
    del  df_distance
file.close()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
  2%|▏         | 3/129 [00:00<00:04, 28.40it/s]

Starting distance calculation...


  0%|          | 0/128 [00:00<?, ?it/s]

Time take for 1 day = 1.0911336223284402 min
Done with Distance/LGA_drop_off/2015-01-01.csv data

Starting distance calculation...


  0%|          | 0/124 [00:00<?, ?it/s]

...

Time take for 1 day = 2.662136709690094 min
Done with Distance/LGA_drop_off/2015-01-29.csv data

Starting distance calculation...


  7%|▋         | 8/111 [00:00<00:01, 78.61it/s]

Time take for 1 day = 3.7520330707232157 min
Done with Distance/LGA_drop_off/2015-01-30.csv data

Starting distance calculation...


100%|██████████| 111/111 [00:32<00:00,  3.37it/s]

Time take for 1 day = 0.5497308135032654 min
Done with Distance/LGA_drop_off/2015-01-31.csv data



# concatenate all days file into month

In [4]:
from calendar import monthrange
import pandas as pd
for folder in ('Distance/LGA_drop_off_months/2015-','Distance/LGA_as_Pickup_months/2015-'):
    for month in range(1,2):
        for days in monthrange(2015, month):
            df_final = pd.DataFrame()
            for day in range(1,days+1,1):
                if day <= 9: 
                    day= '0'+str(day)
                else:
                    day = str(day)
                if month <=9:
                    file = str(folder)+'0'+str(month)+'-'+day+'.csv'
                else:
                    file = str(folder)+str(month)+'-'+day+'.csv'
                
                df = pd.read_csv(file)
                df_final = pd.concat([df_final,df],axis=0)
                if 'drop_off' in folder:
                    destination = 'Distance/LGA_drop_off/2015-'
                else:
                    destination = 'Distance/LGA_as_pickup/2015-'
                df_final.to_csv(str(folder)+str(month)+'.csv')
                print("Done merging {}".format(file))

Done merging Distance/LGA_drop_off/2015-01-01.csv
Done merging Distance/LGA_drop_off/2015-01-02.csv
Done merging Distance/LGA_drop_off/2015-01-03.csv
Done merging Distance/LGA_drop_off/2015-01-01.csv
Done merging Distance/LGA_drop_off/2015-01-02.csv
Done merging Distance/LGA_drop_off/2015-01-03.csv
Done merging Distance/LGA_drop_off/2015-01-04.csv
Done merging Distance/LGA_drop_off/2015-01-05.csv
Done merging Distance/LGA_drop_off/2015-01-06.csv
Done merging Distance/LGA_drop_off/2015-01-07.csv
Done merging Distance/LGA_drop_off/2015-01-08.csv
Done merging Distance/LGA_drop_off/2015-01-09.csv
Done merging Distance/LGA_drop_off/2015-01-10.csv
Done merging Distance/LGA_drop_off/2015-01-11.csv
Done merging Distance/LGA_drop_off/2015-01-12.csv
Done merging Distance/LGA_drop_off/2015-01-13.csv
Done merging Distance/LGA_drop_off/2015-01-14.csv
Done merging Distance/LGA_drop_off/2015-01-15.csv
Done merging Distance/LGA_drop_off/2015-01-16.csv
Done merging Distance/LGA_drop_off/2015-01-17.csv


In [5]:
folder= 'dropsneha'

In [7]:
'sneha' in folder

True